<a href="https://colab.research.google.com/github/sensasi-delight/csbn-test/blob/main/CSBN_REST_API_TEST_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization
Import Required Package

In [1]:
import requests
import json
import pandas
import copy

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_colwidth', None)

handler Function Definition

In [2]:
host_url = "http://128.199.185.208:3000"

column0 = []
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []


def handler(channel, peer, function, data):
  column0.append(copy.deepcopy(channel))
  column1.append(copy.deepcopy(peer['name']))
  column2.append(copy.deepcopy(function))
  column3.append(copy.deepcopy(data))

  if function[:6] == "create":
    url = host_url + "/" + function + "/" + channel + "/" + peer['name']
    response = requests.post(url, json = data).json()
  elif function[:4] == "read":
    url = host_url + "/" + function + "/" + channel + "/" + peer['name'] + "/" + data
    response = requests.get(url).json()
  elif function[:6] == "update":
    url = host_url + "/" + function + "/" + channel + "/" + peer['name'] + "/" + data['id']
    response = requests.put(url, json = data).json()

  column4.append(copy.deepcopy(response))
  column5.append(copy.deepcopy(response['success']))

# The Test

Run Test

In [3]:
# define list of organization and their supplier
organiztions = [
  {
      'name': 'supp1',
      'supplierName': '', # doesn't has a supplier
  }, {
      'name': 'supp2',
      'supplierName': '', # doesn't has a supplier
  }, {
      'name': 'supp3',
      'supplierName': 'supp2',
  }, {
      'name': 'cust1',
      'supplierName': 'supp1',
  }, {
      'name': 'cust2',
      'supplierName': 'supp2',
  }, {
      'name': 'cust3',
      'supplierName': 'supp3',
  }, {
      'name': 'courier',
      'supplierName': '', # doesn't has a supplier
  }
]

# define list of available channel
channels = [
  'channel0',
  'channel1',
  'channel2',
  'channel3',
  'channel4'
]

In [4]:
# loop trough all organizations and channels
for channel in channels:
  for organization in organiztions:

    # define product data
    PRODUCT = {
      "id": "123456",
      "name": "Dummy Data of Product",
      "ingredients": [],
      "halalCertificateId": "12345678901234",
      "manufacturerName": organization['name']
    }

    # store data of product
    handler(channel, organization, 'createProduct', PRODUCT)

    # retrive data of product with id = 123456
    handler(channel, organization, 'readProduct', '123456?orgName=' + organization['supplierName'])

    # update name of product from 'Dummy Data of Product' to 'Dummy Data of Product (edited)'
    PRODUCT['name'] = 'Dummy Data of Product (edited)'

    # submit the edited data of the product
    handler(channel, organization, 'updateProduct', PRODUCT)



    # define batch data
    BATCH = {
      "id": "01",
      "date": "2021-01-03",
      "product": PRODUCT,
      "qty": 1000,
      "ingredients": [],
      "desc": "Dummy Data of Batch"
    }

    # store data of batch
    handler(channel, organization, 'createBatch', BATCH)

    # retrive data of batch with id = 01 and date = 2021-01-03
    handler(channel, organization, 'readBatch', '01?orgName=' + organization['supplierName'] + '&date=2021-01-03')

    # update description of batch from 'Dummy Data of Batch' to 'Dummy Data of Batch (edited)'
    BATCH['desc'] = 'Dummy Data of Batch (edited)'

    # submit the edited data of the batch
    handler(channel, organization, 'updateBatch', BATCH)



    # define invoice data
    INVOICE = {
      "id": "001",
      "date": "2021-01-05",
      "items": [{"batch": BATCH['id'], "qty": 100}],
      "desc": "Dummy Data of Invoice",
    }

    # store data of invoice
    handler(channel, organization, 'createInvoice', INVOICE)

    # retrive data of invoice with id = 001 and date = 2021-01-05
    handler(channel, organization, 'readInvoice', '001?orgName=' + organization['supplierName'] + '&date=2021-01-05')

    # update description of invoice from 'Dummy Data of Invoice' to 'Dummy Data of Invoice (edited)'
    INVOICE['desc'] = 'Dummy Data of Invoice (edited)'

    # submit the edited data of the invoice
    handler(channel, organization, 'updateInvoice', INVOICE)


    # define shipment data
    SHIPMENT = {
      "id": "0001",
      "date": "2021-01-05",
      "desc": "Dummy Data of Shipment",
      "invoices": [INVOICE],
      "history": [
        {
          "date": "2021-01-05",
          "time": "10:00",
          "location": "Pick up at origin"
        }, {
          "date": "2021-01-05",
          "time": "16:00",
          "location": "Arrived at destination"
        }
      ]
    }

    # store data of shipment
    handler(channel, organization, 'createShipment', SHIPMENT)

    # retrive data of shipment with id = 0001 and date = 2021-01-05
    handler(channel, organization, 'readShipment', '0001?orgName=' + organization['supplierName'] + '&date=2021-01-05')

    # update description of shipment from 'Dummy Data of Shipment' to 'Dummy Data of Shipment (edited)'
    SHIPMENT['desc'] = 'Dummy Data of Shipment (edited)'

    # submit the edited data of the shipment
    handler(channel, organization, 'updateShipment', SHIPMENT)


Print result

In [5]:
result = {
  'Channel': column0,
  'Peer': column1,
  'Function': column2,
  'Data Sent': column3,
  'Response': column4,
  'Is Success': column5
}

df = pandas.DataFrame (result, columns = ['Channel', 'Peer', 'Function', 'Data Sent', 'Response', 'Is Success'])
df.to_csv('result.csv')

display(df)

,Channel,Peer,Function,Data Sent,Response,Is Success
0,channel0,supp1,createProduct,"{'id': '123456', 'name': 'Dummy Data of Product', 'ingredients': [], 'halalCertificateId': '12345678901234', 'manufacturerName': 'supp1'}","{'success': True, 'message': 'create Product is success'}",True
1,channel0,supp1,readProduct,123456?orgName=,"{'success': True, 'message': 'read Product is success', 'data': {'id': '123456', 'name': 'Dummy Data of Product', 'ingredients': [], 'halalCertificateId': '12345678901234', 'manufacturerName': 'supp1'}}",True
2,channel0,supp1,updateProduct,"{'id': '123456', 'name': 'Dummy Data of Product (edited)', 'ingredients': [], 'halalCertificateId': '12345678901234', 'manufacturerName': 'supp1'}","{'success': True, 'message': 'update Product is success'}",True
3,channel0,supp1,createBatch,"{'id': '01', 'date': '2021-01-03', 'product': {'id': '123456', 'name': 'Dummy Data of Product (edited)', 'ingredients': [], 'halalCertificateId': '12345678901234', 'manufacturerName': 'supp1'}, 'qty': 1000, 'ingredients': [], 'desc': 'Dummy Data of Batch'}","{'success': True, 'message': 'create Batch is success'}",True
4,channel0,supp1,readBatch,01?orgName=&date=2021-01-03,"{'success': True, 'message': 'read Batch is success', 'data': {'id': '01', 'date': '2021-01-03', 'product': {'id': '123456', 'name': 'Dummy Data of Product (edited)', 'ingredients': [], 'halalCertificateId': '12345678901234', 'manufacturerName': 'supp1'}, 'qty': 1000, 'ingredients': [], 'desc': 'Dummy Data of Batch'}}",True
5,channel0,supp1,updateBatch,"{'id': '01', 'date': '2021-01-03', 'product': {'id': '123456', 'name': 'Dummy Data of Product (edited)', 'ingredients': [], 'halalCertificateId': '12345678901234', 'manufacturerName': 'supp1'}, 'qty': 1000, 'ingredients': [], 'desc': 'Dummy Data of Batch (edited)'}","{'success': True, 'message': 'update Batch is success'}",True
6,channel0,supp1,createInvoice,"{'id': '001', 'date': '2021-01-05', 'items': [{'batch': '01', 'qty': 100}], 'desc': 'Dummy Data of Invoice'}","{'success': False, 'message': 'No valid responses from any peers. Errors: peer=peer0.supp2.csbn.net:30251, status=500, message=error in simulation: transaction returned with failure: Error: Forbidden action: can't execute createInvoice on channel0 peer=peer0.cust1.csbn.net:40151, status=500, message=error in simulation: transaction returned with failure: Error: Forbidden action: can't execute createInvoice on channel0 peer=peer0.cust2.csbn.net:40251, status=500, message=error in simulation: transaction returned with failure: Error: Forbidden action: can't execute createInvoice on channel0 peer=peer0.supp1.csbn.net:30151, status=500, message=error in simulation: transaction returned with failure: Error: Forbidden action: can't execute createInvoice on channel0'}",False
7,channel0,supp1,readInvoice,001?orgName=&date=2021-01-05,"{'success': False, 'message': 'error in simulation: transaction returned with failure: Error: Forbidden action: can't execute readInvoice on channel0'}",False
8,channel0,supp1,updateInvoice,"{'id': '001', 'date': '2021-01-05', 'items': [{'batch': '01', 'qty': 100}], 'desc': 'Dummy Data of Invoice (edited)'}","{'success': False, 'message': 'No valid responses from any peers. Errors: peer=peer0.cust3.csbn.net:40351, status=500, message=error in simulation: transaction returned with failure: Error: Forbidden action: can't execute updateInvoice on channel0 peer=peer0.supp1.csbn.net:30151, status=500, message=error in simulation: transaction returned with failure: Error: Forbidden action: can't execute updateInvoice on channel0 peer=peer0.supp2.csbn.net:30251, status=500, message=error in simulation: transaction returned with failure: Error: Forbidden action: can't execute updateInvoice on channel0 peer=peer0.cust1.csbn.net:40151, status=500, message=error in simulation: transaction returned with failure: Error: Forbidden action: can't execute updateInvoice on channel0'}",False
9,channel0,supp1,createShipment,"{'id': '0001',